In [5]:
import pandas as pd
from datetime import timedelta

# --- Configuration ---
input_filename = 'data.csv' # 你的文件名
output_filename = '2_processed.csv'
time_threshold_minutes = 30
timestamp_format = '%Y/%m/%d %H:%M' # 确认这个格式

# --- 定义合理的日期范围 ---
# 根据你的实际数据情况调整这个范围，比如项目开始和结束的大致年份
REASONABLE_START_DATE = pd.Timestamp('2016-03-10')
REASONABLE_END_DATE = pd.Timestamp('2080-10-02') # 假设数据不会超出这个范围

# --- 1. Read the CSV ---
try:
    df = pd.read_csv(input_filename)
    print(f"Successfully read {input_filename}")
except FileNotFoundError:
    print(f"Error: Input file '{input_filename}' not found.")
    exit()
except Exception as e:
    print(f"Error reading CSV file: {e}")
    exit()

# --- Data Preparation ---
original_row_count = len(df)
print(f"Original row count: {original_row_count}")

# Convert timestamp to datetime objects, coercing errors
df['timestamp'] = pd.to_datetime(df['timestamp'], format=timestamp_format, errors='coerce')

# 检查并丢弃无效时间戳 (NaT)
invalid_timestamps = df['timestamp'].isnull()
num_invalid = invalid_timestamps.sum()

if num_invalid > 0:
    print(f"警告: 发现 {num_invalid} 行的时间戳格式无效或日期不存在 (根据格式 '{timestamp_format}')。")
    df = df.dropna(subset=['timestamp'])
    print(f"已丢弃 {num_invalid} 行无效时间戳数据。剩余行数: {len(df)}")
else:
    print("所有时间戳均成功解析。")

if df.empty:
    print("错误：丢弃无效时间戳后，没有剩余的有效数据。程序退出。")
    exit()

# --- !!! 新增：检查并过滤超出合理范围的日期 !!! ---
rows_before_filter = len(df)
# 找出转换后的最早和最晚日期，用于调试
min_ts_after_conversion = df['timestamp'].min()
max_ts_after_conversion = df['timestamp'].max()
print(f"DEBUG: 转换后并丢弃NaT后的最早时间戳: {min_ts_after_conversion}")
print(f"DEBUG: 转换后并丢弃NaT后的最晚时间戳: {max_ts_after_conversion}")

# 执行过滤
df_filtered = df[(df['timestamp'] >= REASONABLE_START_DATE) & (df['timestamp'] <= REASONABLE_END_DATE)]
rows_after_filter = len(df_filtered)
num_filtered_out = rows_before_filter - rows_after_filter

if num_filtered_out > 0:
    print(f"警告: 进一步过滤掉了 {num_filtered_out} 行，因为它们的时间戳超出了定义的合理范围 ({REASONABLE_START_DATE} 到 {REASONABLE_END_DATE})。")
    df = df_filtered # 使用过滤后的 DataFrame
    if df.empty:
        print("错误：在过滤掉超出范围的日期后，没有剩余的有效数据。程序退出。")
        exit()
else:
    print("所有有效时间戳都在定义的合理范围内。")


# --- 后续处理使用过滤后的 df ---

# IMPORTANT: 在过滤无效行和异常范围行之后，按有效的 timestamp 排序
df = df.sort_values(by='timestamp').reset_index(drop=True)
print("已按有效且在合理范围内的时间戳对数据进行排序。")

# --- 2. Recalculate session_id ('new_session_id') ---
threshold = timedelta(minutes=time_threshold_minutes)
time_diff = df['timestamp'].diff()
new_session_starts = time_diff > threshold
df['new_session_id'] = new_session_starts.cumsum() + 1
print("计算完成 'new_session_id'.")

# --- 3. Calculate session_sequence ---
stage_changed = df['stage'] != df['stage'].shift(1)
session_id_changed = df['new_session_id'] != df['new_session_id'].shift(1)
new_sequence_starts = (stage_changed | session_id_changed).fillna(True)
df['session_sequence'] = new_sequence_starts.cumsum()
print("计算完成 'session_sequence'.")

# --- 4. Group, Aggregate and Merge ---
print("正在分组和聚合数据...")

agg_funcs = {
    'message': ''.join,
    'timestamp': ['min', 'max']
}

grouped = df.groupby(
    ['case_id', 'style', 'stage', 'new_session_id', 'session_sequence', 'speaker'],
    as_index=False
).agg(agg_funcs)

grouped.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in grouped.columns.values]

grouped = grouped.rename(columns={
    'timestamp_min': 'start_time',
    'timestamp_max': 'end_time',
    'message_join': 'message' # 适配 pandas 可能生成的列名
})
if 'message_join' in grouped.columns and 'message' not in grouped.columns:
     grouped = grouped.rename(columns={'message_join': 'message'})

sequence_times = grouped.groupby(['case_id', 'style', 'stage', 'new_session_id', 'session_sequence']).agg(
    start_time=('start_time', 'min'),
    end_time=('end_time', 'max')
).reset_index()

grouped = grouped.drop(columns=['start_time', 'end_time'])
result_df = pd.merge(
    grouped,
    sequence_times,
    on=['case_id', 'style', 'stage', 'new_session_id', 'session_sequence']
)

# --- Final Touches ---
final_columns = [
    'case_id', 'speaker', 'message', 'start_time', 'end_time',
    'new_session_id', 'style', 'stage', 'session_sequence'
]
missing_cols = [col for col in final_columns if col not in result_df.columns]
if missing_cols:
    print(f"警告: 处理后缺少预期的列: {missing_cols}")
    final_columns = [col for col in final_columns if col in result_df.columns]

result_df = result_df[final_columns]
result_df = result_df.sort_values(by=['new_session_id', 'session_sequence', 'speaker']).reset_index(drop=True)

# --- 5. Save the result ---
try:
    # 检查最终结果的日期范围
    final_min_ts = result_df['start_time'].min()
    final_max_ts = result_df['end_time'].max()
    print(f"\n--- 最终结果日期范围 ---")
    print(f"最终最早开始时间: {final_min_ts}")
    print(f"最终最晚结束时间: {final_max_ts}")

    result_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"\n成功处理数据并保存到 {output_filename}")
except Exception as e:
    print(f"\n写入输出 CSV 文件时出错: {e}")

print("\n--- 最终 DataFrame (前5行) ---")
print(result_df.head())
print("\n--- 最终 DataFrame 信息 ---")
result_df.info()

/var/folders/n7/h8vtfbcn3cn8b86gk1fgql700000gn/T/ipykernel_89698/1505262811.py:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_filename)


Successfully read data.csv
Original row count: 273812
警告: 发现 556 行的时间戳格式无效或日期不存在 (根据格式 '%Y/%m/%d %H:%M')。
已丢弃 556 行无效时间戳数据。剩余行数: 273256
DEBUG: 转换后并丢弃NaT后的最早时间戳: 2018-12-27 22:17:00
DEBUG: 转换后并丢弃NaT后的最晚时间戳: 2066-08-08 15:02:00
所有有效时间戳都在定义的合理范围内。
已按有效且在合理范围内的时间戳对数据进行排序。
计算完成 'new_session_id'.
计算完成 'session_sequence'.
正在分组和聚合数据...

--- 最终结果日期范围 ---
最终最早开始时间: 2018-12-27 22:17:00
最终最晚结束时间: 2066-08-08 15:02:00

成功处理数据并保存到 2_processed.csv

--- 最终 DataFrame (前5行) ---
   case_id speaker                                            message  \
0       32       1  谢谢你我冬天也在雪地上走带着滑雪和降落伞没有那么极端。但,是的。我非常喜欢??消息已删除你也...   
1       32       2  我也休息,今天我没有什么特别的事要做。你的冲浪板看起来不错看来你非常喜欢这些极限运动。我经常...   
2       32       1  对你和你的大脑有好处。你好，艾拉，还活着，还有很多事要做吗?嗨，艾拉，我有一个瑞士朋友，他想...   
3       32       2                                        那个时候我已经睡着了。   
4       32       1                                     https://......   

           start_time            end_time  new_session_id  style stage  \
0 2018-12-27 22:17:00 2

In [7]:
import pandas as pd
from datetime import timedelta

# --- Configuration ---
input_filename = 'data.csv' # Your input file name
output_filename = 'output.csv' # Your output file name
time_threshold_minutes = 30
timestamp_format = '%Y/%m/%d %H:%M' # Confirm this format matches your file

# --- 定义合理的日期范围 ---
# 调整起始日期以包含我们修正后的 2016 年数据
REASONABLE_START_DATE = pd.Timestamp('2016-01-01') # Adjusted to include corrected year
REASONABLE_END_DATE = pd.Timestamp('2024-12-31') # Adjust if needed

# --- 1. Read the CSV ---
try:
    df = pd.read_csv(input_filename)
    print(f"Successfully read {input_filename}")
except FileNotFoundError:
    print(f"Error: Input file '{input_filename}' not found.")
    exit()
except Exception as e:
    print(f"Error reading CSV file: {e}")
    exit()

# --- Data Preparation ---
original_row_count = len(df)
print(f"Original row count: {original_row_count}")

# Convert timestamp to datetime objects, coercing errors
df['timestamp'] = pd.to_datetime(df['timestamp'], format=timestamp_format, errors='coerce')

# 检查并丢弃无效时间戳 (NaT)
invalid_timestamps = df['timestamp'].isnull()
num_invalid = invalid_timestamps.sum()

if num_invalid > 0:
    print(f"警告: 发现 {num_invalid} 行的时间戳格式无效或日期不存在 (根据格式 '{timestamp_format}')。")
    df = df.dropna(subset=['timestamp'])
    print(f"已丢弃 {num_invalid} 行无效时间戳数据。剩余行数: {len(df)}")
else:
    print("所有时间戳均成功解析。")

if df.empty:
    print("错误：丢弃无效时间戳后，没有剩余的有效数据。程序退出。")
    exit()

# --- !!! 新增：修正特定年份 (2066 -> 2016) !!! ---
year_to_correct = 2066
target_year = 2016

# 创建一个布尔掩码，标记出需要修正的行
correction_mask = df['timestamp'].dt.year == year_to_correct
num_to_correct = correction_mask.sum()

if num_to_correct > 0:
    print(f"发现 {num_to_correct} 行的时间戳年份为 {year_to_correct}，将修正为 {target_year}...")

    # 定义一个函数来安全地替换年份
    def safe_replace_year(dt_obj, new_year):
        try:
            # pandas timestamp replace handles leap year adjustments correctly
            return dt_obj.replace(year=new_year)
        except ValueError as e:
            # 处理极少数情况，例如尝试将 2月29日 移到非闰年 (虽然 2016 是闰年)
            print(f"  警告: 无法直接替换年份 {dt_obj} -> {new_year}. Error: {e}. 将尝试修正日期。")
            if dt_obj.month == 2 and dt_obj.day == 29:
                try:
                    # 如果目标年不是闰年，将2月29日改为2月28日
                    return dt_obj.replace(year=new_year, day=28)
                except Exception as e_inner:
                    print(f"  修正日期失败: {e_inner}. Setting to NaT.")
                    return pd.NaT # 无法修正，设为 NaT
            else:
                print(f"  未知替换错误. Setting to NaT.")
                return pd.NaT # 其他无法处理的错误

    # 使用 .loc 和 .apply 应用修正函数到被标记的行
    df.loc[correction_mask, 'timestamp'] = df.loc[correction_mask, 'timestamp'].apply(safe_replace_year, new_year=target_year)

    # 再次检查是否有因为修正失败产生的 NaT，并丢弃它们
    post_correction_invalid = df['timestamp'].isnull()
    num_post_invalid = post_correction_invalid.sum() - invalid_timestamps.sum() # 只计算新产生的NaT
    if num_post_invalid > 0:
        print(f"警告：在年份修正过程中，有 {num_post_invalid} 行产生了无效日期 (NaT)，将被丢弃。")
        df = df.dropna(subset=['timestamp'])
        print(f"修正并再次丢弃 NaT 后，剩余行数: {len(df)}")

    print(f"年份修正完成。")

else:
    print(f"未发现需要修正的年份 ({year_to_correct})。")


if df.empty:
    print("错误：在修正年份并丢弃可能的 NaT 后，没有剩余的有效数据。程序退出。")
    exit()


# --- 过滤超出合理范围的日期 (现在包含 2016) ---
rows_before_filter = len(df)
min_ts_after_correction = df['timestamp'].min()
max_ts_after_correction = df['timestamp'].max()
print(f"DEBUG: 年份修正后的最早时间戳: {min_ts_after_correction}")
print(f"DEBUG: 年份修正后的最晚时间戳: {max_ts_after_correction}")

df_filtered = df[(df['timestamp'] >= REASONABLE_START_DATE) & (df['timestamp'] <= REASONABLE_END_DATE)]
rows_after_filter = len(df_filtered)
num_filtered_out = rows_before_filter - rows_after_filter

if num_filtered_out > 0:
    print(f"进一步过滤掉了 {num_filtered_out} 行，因为它们的时间戳超出了定义的合理范围 ({REASONABLE_START_DATE} 到 {REASONABLE_END_DATE})。")
    df = df_filtered
    if df.empty:
        print("错误：在过滤掉超出范围的日期后，没有剩余的有效数据。程序退出。")
        exit()
else:
    print(f"所有有效时间戳都在定义的合理范围内 ({REASONABLE_START_DATE} 到 {REASONABLE_END_DATE})。")


# --- 后续处理使用过滤后的 df ---

# IMPORTANT: 排序
df = df.sort_values(by='timestamp').reset_index(drop=True)
print("已按有效且在合理范围内的时间戳对数据进行排序。")

# --- 2. Recalculate session_id ('new_session_id') ---
threshold = timedelta(minutes=time_threshold_minutes)
time_diff = df['timestamp'].diff()
new_session_starts = time_diff > threshold
df['new_session_id'] = new_session_starts.cumsum() + 1
print("计算完成 'new_session_id'.")

# --- 3. Calculate session_sequence ---
stage_changed = df['stage'] != df['stage'].shift(1)
session_id_changed = df['new_session_id'] != df['new_session_id'].shift(1)
new_sequence_starts = (stage_changed | session_id_changed).fillna(True)
df['session_sequence'] = new_sequence_starts.cumsum()
print("计算完成 'session_sequence'.")

# --- 4. Group, Aggregate and Merge ---
print("正在分组和聚合数据...")

agg_funcs = {
    'message': ''.join,
    'timestamp': ['min', 'max']
}

grouped = df.groupby(
    ['case_id', 'style', 'stage', 'new_session_id', 'session_sequence', 'speaker'],
    as_index=False
).agg(agg_funcs)

grouped.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in grouped.columns.values]

grouped = grouped.rename(columns={
    'timestamp_min': 'start_time',
    'timestamp_max': 'end_time',
    'message_join': 'message' # 适配 pandas 可能生成的列名
})
if 'message_join' in grouped.columns and 'message' not in grouped.columns:
     grouped = grouped.rename(columns={'message_join': 'message'})

sequence_times = grouped.groupby(['case_id', 'style', 'stage', 'new_session_id', 'session_sequence']).agg(
    start_time=('start_time', 'min'),
    end_time=('end_time', 'max')
).reset_index()

grouped = grouped.drop(columns=['start_time', 'end_time'])
result_df = pd.merge(
    grouped,
    sequence_times,
    on=['case_id', 'style', 'stage', 'new_session_id', 'session_sequence']
)

# --- Final Touches ---
final_columns = [
    'case_id', 'speaker', 'message', 'start_time', 'end_time',
    'new_session_id', 'style', 'stage', 'session_sequence'
]
missing_cols = [col for col in final_columns if col not in result_df.columns]
if missing_cols:
    print(f"警告: 处理后缺少预期的列: {missing_cols}")
    final_columns = [col for col in final_columns if col in result_df.columns]

result_df = result_df[final_columns]
result_df = result_df.sort_values(by=['new_session_id', 'session_sequence', 'speaker']).reset_index(drop=True)

# --- 5. Save the result ---
try:
    final_min_ts = result_df['start_time'].min()
    final_max_ts = result_df['end_time'].max()
    print(f"\n--- 最终结果日期范围 ---")
    print(f"最终最早开始时间: {final_min_ts}")
    print(f"最终最晚结束时间: {final_max_ts}")

    result_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"\n成功处理数据并保存到 {output_filename}")
except Exception as e:
    print(f"\n写入输出 CSV 文件时出错: {e}")

print("\n--- 最终 DataFrame (前5行) ---")
print(result_df.head())
print("\n--- 最终 DataFrame 信息 ---")
result_df.info()

/var/folders/n7/h8vtfbcn3cn8b86gk1fgql700000gn/T/ipykernel_89698/3150600587.py:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_filename)


Successfully read data.csv
Original row count: 273812
警告: 发现 556 行的时间戳格式无效或日期不存在 (根据格式 '%Y/%m/%d %H:%M')。
已丢弃 556 行无效时间戳数据。剩余行数: 273256
发现 15994 行的时间戳年份为 2066，将修正为 2016...
年份修正完成。
DEBUG: 年份修正后的最早时间戳: 2016-05-24 18:10:00
DEBUG: 年份修正后的最晚时间戳: 2024-06-25 01:16:00
所有有效时间戳都在定义的合理范围内 (2016-01-01 00:00:00 到 2024-12-31 00:00:00)。
已按有效且在合理范围内的时间戳对数据进行排序。
计算完成 'new_session_id'.
计算完成 'session_sequence'.
正在分组和聚合数据...

--- 最终结果日期范围 ---
最终最早开始时间: 2016-05-24 18:10:00
最终最晚结束时间: 2024-06-25 01:16:00

成功处理数据并保存到 output.csv

--- 最终 DataFrame (前5行) ---
   case_id speaker                                      message  \
0       35       1  消息和通话都进行端到端加密对话之外的任何人甚至包含 WhatsApp 都无法读取或收听   
1       35       2                                           你好   
2       35       2                                   你的封面是你的照片吗   
3       35       1                                           是的   
4       35       2     你的照片我看起来非常陌生我们以前似乎没有见过我将我的照片发给你你看一下是否想起我   

           start_time            end_time  new_session_id  s

In [9]:
import pandas as pd
from datetime import timedelta

# --- Configuration ---
input_filename = 'data.csv' # Your input file name
output_filename = 'output_caseid.csv' # Your output file name
time_threshold_minutes = 30
timestamp_format = '%Y/%m/%d %H:%M' # Confirm this format matches your file

# --- 定义合理的日期范围 ---
REASONABLE_START_DATE = pd.Timestamp('2016-01-01') # Adjusted to include corrected year
REASONABLE_END_DATE = pd.Timestamp('2024-12-31') # Adjust if needed

# --- 1. Read the CSV ---
try:
    df = pd.read_csv(input_filename)
    print(f"Successfully read {input_filename}")
except FileNotFoundError:
    print(f"Error: Input file '{input_filename}' not found.")
    exit()
except Exception as e:
    print(f"Error reading CSV file: {e}")
    exit()

# --- Data Preparation ---
original_row_count = len(df)
print(f"Original row count: {original_row_count}")

# Convert timestamp to datetime objects, coercing errors
df['timestamp'] = pd.to_datetime(df['timestamp'], format=timestamp_format, errors='coerce')

# 检查并丢弃无效时间戳 (NaT)
invalid_timestamps = df['timestamp'].isnull()
num_invalid = invalid_timestamps.sum()

if num_invalid > 0:
    print(f"警告: 发现 {num_invalid} 行的时间戳格式无效或日期不存在 (根据格式 '{timestamp_format}')。")
    df = df.dropna(subset=['timestamp'])
    print(f"已丢弃 {num_invalid} 行无效时间戳数据。剩余行数: {len(df)}")
else:
    print("所有时间戳均成功解析。")

if df.empty:
    print("错误：丢弃无效时间戳后，没有剩余的有效数据。程序退出。")
    exit()

# --- 修正特定年份 (2066 -> 2016) ---
year_to_correct = 2066
target_year = 2016
correction_mask = df['timestamp'].dt.year == year_to_correct
num_to_correct = correction_mask.sum()

if num_to_correct > 0:
    print(f"发现 {num_to_correct} 行的时间戳年份为 {year_to_correct}，将修正为 {target_year}...")
    def safe_replace_year(dt_obj, new_year):
        try:
            return dt_obj.replace(year=new_year)
        except ValueError as e:
            print(f"  警告: 无法直接替换年份 {dt_obj} -> {new_year}. Error: {e}. 将尝试修正日期。")
            if dt_obj.month == 2 and dt_obj.day == 29:
                try: return dt_obj.replace(year=new_year, day=28)
                except Exception as e_inner: print(f"  修正日期失败: {e_inner}. Setting to NaT."); return pd.NaT
            else: print(f"  未知替换错误. Setting to NaT."); return pd.NaT

    df.loc[correction_mask, 'timestamp'] = df.loc[correction_mask, 'timestamp'].apply(safe_replace_year, new_year=target_year)
    post_correction_invalid = df['timestamp'].isnull()
    num_post_invalid = post_correction_invalid.sum() - invalid_timestamps.sum() # 计算新产生的NaT
    if num_post_invalid > 0:
        print(f"警告：在年份修正过程中，有 {num_post_invalid} 行产生了无效日期 (NaT)，将被丢弃。")
        df = df.dropna(subset=['timestamp'])
        print(f"修正并再次丢弃 NaT 后，剩余行数: {len(df)}")
    print(f"年份修正完成。")
else:
    print(f"未发现需要修正的年份 ({year_to_correct})。")

if df.empty:
    print("错误：在修正年份并丢弃可能的 NaT 后，没有剩余的有效数据。程序退出。")
    exit()

# --- 过滤超出合理范围的日期 ---
rows_before_filter = len(df)
min_ts_after_correction = df['timestamp'].min()
max_ts_after_correction = df['timestamp'].max()
# print(f"DEBUG: 年份修正后的最早时间戳: {min_ts_after_correction}") # 可以取消注释来调试
# print(f"DEBUG: 年份修正后的最晚时间戳: {max_ts_after_correction}") # 可以取消注释来调试

df_filtered = df[(df['timestamp'] >= REASONABLE_START_DATE) & (df['timestamp'] <= REASONABLE_END_DATE)]
rows_after_filter = len(df_filtered)
num_filtered_out = rows_before_filter - rows_after_filter

if num_filtered_out > 0:
    print(f"进一步过滤掉了 {num_filtered_out} 行，因为它们的时间戳超出了定义的合理范围 ({REASONABLE_START_DATE} 到 {REASONABLE_END_DATE})。")
    df = df_filtered
    if df.empty:
        print("错误：在过滤掉超出范围的日期后，没有剩余的有效数据。程序退出。")
        exit()
else:
    print(f"所有有效时间戳都在定义的合理范围内 ({REASONABLE_START_DATE} 到 {REASONABLE_END_DATE})。")

# --- 后续处理使用过滤后的 df ---
# IMPORTANT: 排序
df = df.sort_values(by=['case_id', 'timestamp']).reset_index(drop=True) # 初始排序也加入case_id
print("已按 case_id 和有效时间戳对数据进行排序。")

# --- 2. Recalculate session_id ('new_session_id') ---
# 在计算 diff 前，确保数据是按时间顺序排列的，但 session_id 的计算逻辑是基于时间差，
# 与 case_id 无关，所以在计算 diff 时仍按时间排序分组内时间。
# 但因为后续 session_sequence 依赖 session_id 和 stage，且分组聚合依赖 case_id，
# 保持按 case_id, timestamp 排序是合理的。diff 计算会跨 case_id，但这符合原始逻辑？
# 重新思考：session_id 应该在同一个 case_id 内计算。
# 我们需要在计算 diff 之前按 case_id 分组。

print("正在计算 'new_session_id' (按 case_id 分组)...")
threshold = timedelta(minutes=time_threshold_minutes)
# 按 case_id 分组计算时间差，避免跨 case 计算
df['time_diff'] = df.groupby('case_id')['timestamp'].diff()
# new_session_starts 条件也要考虑 case_id 的变化 (分组的第一个元素总是新 session)
# shift() 默认按 index 移动，因为我们按 case_id, timestamp 排序了，所以可以直接比较
is_first_in_case = ~df['case_id'].duplicated(keep='first')
new_session_starts = (df['time_diff'] > threshold) | is_first_in_case
# cumsum 在整个 DataFrame 上进行，但因为 is_first_in_case 的存在，能正确为每个 case_id 重置 session 起始
df['new_session_id'] = new_session_starts.cumsum()
# 移除临时的 time_diff 列
df = df.drop(columns=['time_diff'])
print("计算完成 'new_session_id'.")


# --- 3. Calculate session_sequence ---
# session_sequence 的计算逻辑依赖于 stage 和 new_session_id 的变化
print("正在计算 'session_sequence'...")
# shift(1) 默认基于当前的排序（case_id, timestamp）
stage_changed = df['stage'] != df['stage'].shift(1)
session_id_changed = df['new_session_id'] != df['new_session_id'].shift(1)
# 同样，每个 case_id 的第一条记录也应该开始新的 sequence
# is_first_in_case 已经在上面计算过了
new_sequence_starts = (stage_changed | session_id_changed | is_first_in_case).fillna(True) # fillna(True) 处理全局第一行
df['session_sequence'] = new_sequence_starts.cumsum()
print("计算完成 'session_sequence'.")


# --- 4. Group, Aggregate and Merge ---
print("正在分组和聚合数据...")

agg_funcs = {
    'message': ''.join,
    'timestamp': ['min', 'max']
}

# 分组键现在仍然是这些，因为它们定义了一个会话片段
grouped = df.groupby(
    ['case_id', 'style', 'stage', 'new_session_id', 'session_sequence', 'speaker'],
    as_index=False
).agg(agg_funcs)

grouped.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in grouped.columns.values]

grouped = grouped.rename(columns={
    'timestamp_min': 'start_time',
    'timestamp_max': 'end_time',
    'message_join': 'message'
})
if 'message_join' in grouped.columns and 'message' not in grouped.columns:
     grouped = grouped.rename(columns={'message_join': 'message'})

sequence_times = grouped.groupby(['case_id', 'style', 'stage', 'new_session_id', 'session_sequence']).agg(
    start_time=('start_time', 'min'),
    end_time=('end_time', 'max')
).reset_index()

grouped = grouped.drop(columns=['start_time', 'end_time'])
result_df = pd.merge(
    grouped,
    sequence_times,
    on=['case_id', 'style', 'stage', 'new_session_id', 'session_sequence']
)

# --- Final Touches ---
final_columns = [
    'case_id', 'speaker', 'message', 'start_time', 'end_time',
    'new_session_id', 'style', 'stage', 'session_sequence'
]
missing_cols = [col for col in final_columns if col not in result_df.columns]
if missing_cols:
    print(f"警告: 处理后缺少预期的列: {missing_cols}")
    final_columns = [col for col in final_columns if col in result_df.columns]

result_df = result_df[final_columns]

# --- !!! 修改排序规则 !!! ---
# 将 case_id 作为第一排序键
result_df = result_df.sort_values(
    by=['case_id', 'new_session_id', 'session_sequence', 'speaker']
).reset_index(drop=True)
print("最终结果已按 'case_id', 'new_session_id', 'session_sequence', 'speaker' 排序。")


# --- 5. Save the result ---
try:
    final_min_ts = result_df['start_time'].min()
    final_max_ts = result_df['end_time'].max()
    print(f"\n--- 最终结果日期范围 ---")
    print(f"最终最早开始时间: {final_min_ts}")
    print(f"最终最晚结束时间: {final_max_ts}")

    result_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"\n成功处理数据并保存到 {output_filename}")
except Exception as e:
    print(f"\n写入输出 CSV 文件时出错: {e}")

print("\n--- 最终 DataFrame (前5行) ---")
print(result_df.head())
# 打印最后几行，以检查不同 case_id 的排序
print("\n--- 最终 DataFrame (后5行) ---")
print(result_df.tail())
print("\n--- 最终 DataFrame 信息 ---")
result_df.info()

/var/folders/n7/h8vtfbcn3cn8b86gk1fgql700000gn/T/ipykernel_89698/1034096464.py:16: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_filename)


Successfully read data.csv
Original row count: 273812
警告: 发现 556 行的时间戳格式无效或日期不存在 (根据格式 '%Y/%m/%d %H:%M')。
已丢弃 556 行无效时间戳数据。剩余行数: 273256
发现 15994 行的时间戳年份为 2066，将修正为 2016...
年份修正完成。
所有有效时间戳都在定义的合理范围内 (2016-01-01 00:00:00 到 2024-12-31 00:00:00)。
已按 case_id 和有效时间戳对数据进行排序。
正在计算 'new_session_id' (按 case_id 分组)...
计算完成 'new_session_id'.
正在计算 'session_sequence'...
计算完成 'session_sequence'.
正在分组和聚合数据...
最终结果已按 'case_id', 'new_session_id', 'session_sequence', 'speaker' 排序。

--- 最终结果日期范围 ---
最终最早开始时间: 2016-05-24 18:10:00
最终最晚结束时间: 2024-06-25 01:16:00

成功处理数据并保存到 output_caseid.csv

--- 最终 DataFrame (前5行) ---
   case_id speaker                                            message  \
0        1       1  哈喽正在收拾准备下班哪有没事这几个月上班也有点闲那我叫您涵吧Cola学到了销售啊您呢哗利害年...   
1        1       2  哈喽你在干嘛呢下班了吗你的声音真的很可爱辛苦了这么晚认真的我叫丁涵，来自贵阳很开心认识你哈哈...   
2        1       1  嗯嗯未呀睇youtube我还早著呢您差不多休息了吧我12至2所以您保养得好好哦还可以我昨天还...   
3        1       2  我好了你睡了吗在干什么呢这么晚我快了哈哈你几点睡呢 哈哈哈你睡这么晚白天工作有精神吗想我吗哈...   
4        1       1              